<a href="https://colab.research.google.com/github/baiyunming/I2DL-WS2020/blob/main/trainYOLO_map_model_whole_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from google.colab import drive
from torch.utils.data import DataLoader

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ls

drive/  sample_data/


In [4]:
%cd drive/MyDrive/ObjectDetection/

/content/drive/MyDrive/ObjectDetection


In [5]:
from model import YOLO_Resnet
from loss import YoloLoss
from animal_dataset import AnimalDataset

In [6]:
train_dataset = AnimalDataset("/content/drive/MyDrive/ObjectDetection/animal_dataset/train.csv")

In [7]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
img, label = next(iter(train_dataloader))
print(img.shape)
print(label.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64, 5, 5, 14])


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO_Resnet().to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [9]:
prediction = model(img.to(device))

In [10]:
prediction.shape

torch.Size([64, 350])

In [11]:
loss = YoloLoss()

In [12]:
m = loss(prediction, label.to(device))
print(m/64)

tensor(237.4242, device='cuda:0', grad_fn=<DivBackward0>)


In [13]:
import torch.optim as optim
m.backward()
optimizer = optim.Adam(model.parameters(), lr=2e-5)
optimizer.step()


In [14]:
prediction_new = model(img.to(device)) 
m = loss(prediction_new, label.to(device))
print(m/64)

tensor(200.6017, device='cuda:0', grad_fn=<DivBackward0>)


In [16]:
prediction_new.shape 

torch.Size([64, 350])

In [41]:
pred_box1 = prediction[...,5:9]
pred_box2 = prediction[...,10:14]

In [107]:
pred_box1.shape

torch.Size([64, 5, 5, 4])

In [45]:
cat_result = torch.cat([(prediction[...,4]).unsqueeze(0), prediction[...,9].unsqueeze(0)],dim=0)

In [51]:
box_high_score = cat_result.argmax(0).unsqueeze(-1)

In [52]:
box_high_score.shape #if zero first box has higher confidence, if one second box has higher confidence

torch.Size([64, 5, 5, 1])

In [54]:
final_box =(1-box_high_score) * pred_box1 + box_high_score * pred_box2

In [57]:
final_box.shape

torch.Size([64, 5, 5, 4])

In [99]:
#convert x, y width, height of box w.r.t. whole image
S = 5 
idx_x = torch.arange(S).repeat(batch_size,S,1)

In [100]:
idx_y = torch.arange(S).unsqueeze(-1).repeat(batch_size,1,S)

In [101]:
idx_y[1,:,:]

tensor([[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]])

In [103]:
idx_x[1,:,:]

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])

In [129]:
# dim: batch_size * S * S * 1 
x_wrt_image = (1/S) * (final_box[...,0:1] + idx_x.unsqueeze(-1)) 
y_wrt_image = (1/S) * (final_box[...,1:2] + idx_y.unsqueeze(-1)) 
width_wrt_image = (1/S) * final_box[...,2:3]
height_wrt_image = (1/S) * final_box[...,3:4]
clss_label = prediction[...,0:4].argmax(-1).unsqueeze(-1) #get class label
confidence = torch.max(prediction[...,4], prediction[...,9]).unsqueeze(-1)

In [118]:
y_wrt_image.shape

torch.Size([64, 5, 5, 1])

In [126]:
torch.max(prediction[...,4], prediction[...,9]).shape

torch.Size([64, 5, 5])

In [173]:
#batch_size * S * S * 6 (class, confidence, box_wrt_image)
conerted_box = torch.cat([clss_label, confidence, x_wrt_image, y_wrt_image, width_wrt_image, height_wrt_image],dim=-1)

In [175]:
conerted_box.shape

torch.Size([64, 5, 5, 6])

In [161]:
label[1,:,:,10] #first sample, 5*5, 10th element

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [176]:
reshaped_box = conerted_box.reshape(batch_size, S*S, -1) #batch_size, S*S, 6 [class_idx, object_probability, x, y, w, h]

In [189]:
reshaped_box.shape

torch.Size([64, 25, 6])

In [168]:
reshaped_box

tensor([3])

In [221]:
boxes_batchs = []
for i in range(batch_size):
  boxes_frame = []
  for j in range(S*S):
    boxes_frame.append([x.item() for x in reshaped_box[i,j,:]])
  boxes_batchs.append(boxes_frame)  


In [17]:
def get_list_boxes(prediction, S=5):
  #prediction dim: batch_size * S * S * 14 （class_idx 4 + 5 * B=2）
  #return list: batch_size * (S*S) 
  batch_size = prediction.shape[0]
  prediction = prediction.to("cpu")
  prediction = prediction.reshape(prediction.shape[0], S, S, 14)
  pred_box1 = prediction[...,5:9]
  pred_box2 = prediction[...,10:14]

  cat_result = torch.cat([(prediction[...,4]).unsqueeze(0), prediction[...,9].unsqueeze(0)],dim=0)
  box_high_score = cat_result.argmax(0).unsqueeze(-1)
  final_box =(1-box_high_score) * pred_box1 + box_high_score * pred_box2

  idx_x = torch.arange(S).repeat(batch_size,S,1)
  idx_y = torch.arange(S).unsqueeze(-1).repeat(batch_size,1,S)

  # dim: batch_size * S * S * 1 
  x_wrt_image = (1/S) * (final_box[...,0:1] + idx_x.unsqueeze(-1)) 
  y_wrt_image = (1/S) * (final_box[...,1:2] + idx_y.unsqueeze(-1)) 
  width_wrt_image = (1/S) * final_box[...,2:3]
  height_wrt_image = (1/S) * final_box[...,3:4]
  clss_label = prediction[...,0:4].argmax(-1).unsqueeze(-1) #get class label
  confidence = torch.max(prediction[...,4], prediction[...,9]).unsqueeze(-1)

  converted_box = torch.cat([clss_label, confidence, x_wrt_image, y_wrt_image, width_wrt_image, height_wrt_image],dim=-1)

  reshaped_box = converted_box.reshape(batch_size, S*S, -1) #batch_size, S*S, 6 [class_idx, object_probability, x, y, w, h]
  print(reshaped_box.shape)

  boxes_batch = []
  for i in range(batch_size):
    boxes_frame = []
    for j in range(S*S):
      boxes_frame.append([x.item() for x in reshaped_box[i,j,:]])
    boxes_batch.append(boxes_frame)

  
  return boxes_batch 


In [215]:
mmm = get_list_boxes(prediction)

torch.Size([64, 25, 6])


In [225]:
prediction.shape

torch.Size([64, 5, 5, 14])

In [20]:
model.eval()
all_predict_boxes = []
all_true_boxes = []
frame_idx = 0
iou_threshold = 0.5
confidence_threshold = 0.3


for batch_idx, (image, labels) in enumerate(train_dataloader):
  with torch.no_grad():
    prediction = model(image.to(device))
  
  predict_boxes_batch = get_list_boxes(prediction, S=5)
  true_boxes_batch = get_list_boxes(labels.to(device), S=5)
  batch_size = prediction.shape[0]

  for i in range(batch_size):
    predict_boxes_frame = predict_boxes_batch[i]
    nms_predict_boxes_frame = non_max_suppression(predict_boxes_frame,iou_threshold,confidence_threshold)

    for j in range(len(nms_predict_boxes_frame)):
      all_predict_boxes.append([frame_idx] + nms_predict_boxes_frame[j])

    true_boxes_frame = true_boxes_batch[i]
    for k in range(len(true_boxes_frame)):
      if true_boxes_frame[k][1] > confidence_threshold:
        all_true_boxes.append([frame_idx] + true_boxes_frame[k])
    frame_idx = frame_idx + 1



torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([64, 25, 6])
torch.Size([47, 25, 6])
torch.Size([47, 25, 6])


In [22]:
from utils import mean_average_precision

In [18]:
from utils import non_max_suppression 

In [23]:
mean_average_precision(all_predict_boxes, all_true_boxes)

tensor(4.9056e-08)

In [235]:
nms_predict_boxes = non_max_suppression(mmm[0], 0.5, probability_threshold=0.3)

In [236]:
len(nms_predict_boxes)

15

In [239]:
aaa = get_list_boxes(label.to(device), S=5)

torch.Size([64, 25, 6])


In [250]:
aaa[0][5]

[0.0, 0.0, 0.0, 0.20000000298023224, 0.0, 0.0]